1. Write a python program which searches all the product under a particular product from www.amazon.in.
The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search
for guitars.

In [ ]:
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
import time
import re
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def search_prod_amazon(search_str):
    #intiating the browser
    driver=webdriver.Chrome('C:/Users/yn/Desktop/Yuvi/DataTrained/seleniumWebdriver/chromedriver.exe')
    #loading the given url
    driver.get('https://www.amazon.in/')
    #Send input to search bar
    time.sleep(5)
    search_bar= WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, '//input[@name="field-keywords"]')))
    search_bar.send_keys(search_str)
    driver.find_element_by_id("nav-search-submit-button").click()
    time.sleep(10)
    return driver

2. In the above question, now scrape the following details of each product listed in first 3 pages of your
search results and save it in a data frame and csv. In case if any product has less than 3 pages in search
results then scrape all the products available under that product name. Details to be scraped are: "Brand
Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and
“Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [ ]:
def get_prod_info(input_string):
    driver=search_prod_amazon(input_string)
    driver.maximize_window()    
    list_items=[]
    brand_list=[]
    name_list=[]
    price_list=[]
    return_list=[]
    exp_delvry=[]
    availb_list=[]
    prod_url=[]
    ret_exchng_list=[]
    
    for i in range(1,4):
        #WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.XPATH, '//div[@data-component-type="s-search-result"]//a[@class="a-link-normal s-link-style a-text-normal"]')))
        list_items=driver.find_elements_by_xpath('//div[@data-component-type="s-search-result"]')
        for item in list_items:
            try:
                brand_list.append(item.find_element_by_xpath('.//span[@class="a-size-base-plus a-color-base"]').text)
            except:
                brand_list.append('-')
            try:
                name_list.append(item.find_element_by_xpath('.//h2[1]//span').text)
            except:
                name_list.append('-')
            try:
                price_list.append(item.find_element_by_xpath('.//span[@class="a-price-whole"]/parent::span[1]').text)
            except:
                price_list.append('-')
            try:
                exp_delvry.append(item.find_element_by_xpath('.//span[starts-with(.,"Get it by")]/span[2]').text)
            except:
                exp_delvry.append('-')
            try:
                availb_list.append(item.find_element_by_xpath('.//span[@class="a-size-base a-color-price"]').text)
            except:
                availb_list.append('-')
            try:
                prod_url.append(item.find_element_by_xpath('.//a[@class="a-link-normal s-link-style a-text-normal" or @class="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"]').get_attribute('href'))
            except:
                prod_url.append('-')
        driver.execute_script("window.open('');")
        driver.switch_to.window(driver.window_handles[1])
        for urls in prod_url:
            try:
                driver.get(urls)
                time.sleep(4)
                ret_exchng_list.append(driver.find_element_by_xpath('//div[@data-name="RETURNS_POLICY"]//a[@class="a-size-small a-link-normal a-text-normal"]').text)
            except:
                ret_exchng_list.append('-')
        # Closing new_url tab
        driver.close()
        # Switching to old tab
        driver.switch_to.window(driver.window_handles[0])
        try:
            driver.find_element_by_xpath('//a[.="Next→" or .="Next"]').click()
        except NoSuchElementException:
            driver.refresh()
            time.sleep(5)
            driver.find_element_by_xpath('//a[.="Next→" or .="Next"]').click()
    data_collection=pd.DataFrame({'Brand Name':brand_list,'Name of the Product':name_list,'Price':price_list,'Expected Delivery':exp_delvry,'Availability':availb_list,'Product URL':prod_url,'Return/Exchange':ret_exchng_list})
    return data_collection
    

In [ ]:
get_prod_info('Jerkins')


3. Write a python program to access the search bar and search button on images.google.com and scrape 10
images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’.


In [ ]:
def google_images(search_str_list):
    #intiating the browser
    driver=webdriver.Chrome('C:/Users/yn/Desktop/Yuvi/DataTrained/seleniumWebdriver/chromedriver.exe')
    #loading the given url
    driver.get('https://images.google.com/')
    driver.maximize_window()
    #Send input to search bar
    time.sleep(5)
    image_urls=[]
    WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, '//input[@title="Search"]')))
    for key in search_str_list:
        driver.find_element_by_xpath('//input[@title="Search"]').send_keys(key)
        driver.find_element_by_xpath('//button[@aria-label="Google Search"]').click()
        time.sleep(10)
        #try:
        images=driver.find_elements_by_xpath('//div[@class="islrc"]/div/a[@jsname="sTFXNd"]//img[1]')
        print(len(images))
        for image in images[:10]:
            try:
                image_urls.append(image.get_attribute('src').strip())
            except:
                image_urls.append('-')
        driver.find_element_by_xpath('//input[@title="Search"]').clear()
        print(key,' is done.')
    result=pd.DataFrame({'Image Urls':image_urls})
    return result

In [ ]:
search_keys=['fruits','cars','Machine Learning','Guitar','Cakes']
google_images(search_keys)

4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on
www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be
scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”,
“Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the
details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [ ]:
def search_prod_flipkart(search_str):
    #intiating the browser
    driver=webdriver.Chrome('C:/Users/yn/Desktop/Yuvi/DataTrained/seleniumWebdriver/chromedriver.exe')
    #loading the given url
    driver.get('https://www.flipkart.com/')
    #Send input to search bar
    time.sleep(5)
    driver.find_element_by_xpath('//button[@class="_2KpZ6l _2doB4z"]').click()
    search_bar= WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, '//input[@title="Search for products, brands and more"]')))
    driver.find_element_by_xpath('//input[@title="Search for products, brands and more"]').send_keys(search_str)
    driver.find_element_by_xpath('//button[@type="submit"]').click()
    time.sleep(5)
    brand_list=[]
    name_list=[]
    color_list=[]
    ram_list=[]
    rom_list=[]
    p_cam_list=[]
    s_cam_list=[]
    disp_size_list=[]
    btry_cap_list=[]
    price_list=[]
    url_list=[]
    
    list_item=driver.find_elements_by_xpath('//div[@class="_13oc-S"]/div')
    for item in list_item:
        brand_list.append('-')
        try:
            name=item.find_element_by_xpath('.//div[@class="_4rR01T"]').text
            name_list.append(name)
            color=name[name.index('(')+1:]
            color=color[:color.index(',')]
            color_list.append(color)
        except:
            name_list.append('-')
            color_list.append('-')        
        try:
            ram_str=item.find_element_by_xpath('.//ul[@class="_1xgFaf"]/li[contains(.,"RAM")]').text
            ram=ram_str.split('|')[0]
            rom=ram_str.split('|')[1]
            ram_list.append(ram)
            rom_list.append(rom)
        except:
            ram_list.append('-')
            rom_list.append('-')
        try:
            cam_str=item.find_element_by_xpath('.//ul[@class="_1xgFaf"]/li[contains(.,"Camera")]').text
            f_cam=cam_str.split('|')[0]
            s_cam=cam_str.split('|')[1]
            p_cam_list.append(f_cam)
            s_cam_list.append(s_cam)
        except:
            p_cam_list.append('-')
            s_cam_list.append('-')
        try:
            dis_str=item.find_element_by_xpath('.//ul[@class="_1xgFaf"]/li[contains(.,"Display")]').text
            disp_size_list.append(dis_str)
        except:
            disp_size_list.append('-')
        try:
            btry=item.find_element_by_xpath('.//ul[@class="_1xgFaf"]/li[contains(.,"Battery")]').text
            btry_cap_list.append(btry)
        except:
            btry_cap_list.append('-')
        try:
            price=item.find_element_by_xpath('.//div[@class="_30jeq3 _1_WHN1"]').text
            price_list.append(price)
        except:
            price_list.append('-')
        try:
            url=item.find_element_by_xpath('.//a[1]').get_attribute('href')
            url_list.append(url)
        except:
            url_list.append('-')
    result=pd.DataFrame({'Brand':brand_list,'Name':name_list,'Color':color_list,'Price':price_list,'RAM':ram_list,'ROM':rom_list,'Battery':btry_cap_list,'Display':disp_size_list,'Primary Camera':p_cam_list,'Secondary Camera':s_cam_list,'Url':url_list})
    result.to_csv('C:/Users/yn/Desktop/Yuvi/DataTrained/Internship/flipkartProdDetails.csv')
    return result

In [ ]:
search_prod_flipkart('Moto G5')

5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google
maps.

In [ ]:
def find_lat_long_googlemaps(search_str):
    #intiating the browser
    driver=webdriver.Chrome('C:/Users/yn/Desktop/Yuvi/DataTrained/seleniumWebdriver/chromedriver.exe')
    #loading the given url
    driver.get('https://www.google.com/maps/')
    #Send input to search bar
    time.sleep(5)
    driver.find_element_by_xpath('//input[@id="searchboxinput"]').clear()
    driver.find_element_by_xpath('//input[@id="searchboxinput"]').send_keys(search_str)
    driver.find_element_by_xpath('//button[@id="searchbox-searchbutton"]').click()
    time.sleep(4)
    curl=driver.current_url
    ptrn=re.compile('https://www.google.com/maps/place/.*/@(.*?)/')
    res=re.search(ptrn,curl)
    longitude=res.group(1).split(',')[1]
    latitude=res.group(1).split(',')[0]
    print("Longitude: ",longitude,"Lattitude: ",latitude)

In [ ]:
find_lat_long_googlemaps('Bangalore,Karnataka')

6. Write a program to scrap details of all the funding deals for second quarter (i.e Jan 21 – March 21)
from trak.in

In [ ]:
def get_funding_deals():
    #intiating the browser
    options = Options()
    options.add_argument("--disable-notifications")
    
    driver=webdriver.Chrome('C:/Users/yn/Desktop/Yuvi/DataTrained/seleniumWebdriver/chromedriver.exe',chrome_options=options)
    #loading the given url
    driver.get('https://trak.in/')
    #Send input to search bar
    time.sleep(5)
    driver.maximize_window()
    driver.find_element_by_xpath('(//a[.="Funding Deals"])[1]').click()
    time.sleep(5)
    date_list=[]
    startup_list=[]
    industry_vertical_list=[]
    sub_vertical_list=[]
    city_location_list=[]
    investors_name_list=[]
    investment_type_list=[]
    amount_list=[]
    
    try:
        tables_list=driver.find_elements_by_xpath('//h2[.="January, 2021" or .="February, 2021" or .="March, 2021"]/following-sibling::div[1]//div[@class="dataTables_scrollBody"]/table[1]')
    except:
        driver.refresh()
        time.sleep(6)
        tables_list=driver.find_elements_by_xpath('//h2[.="January, 2021" or .="February, 2021" or .="March, 2021"]/following-sibling::div[1]//div[@class="dataTables_scrollBody"]/table[1]') 
    print(len(tables_list))
    for table in tables_list:
        head_tags=table.find_element_by_xpath('.//thead/tr')
        child=head_tags.find_elements_by_xpath('//th')
        date=head_tags.find_element_by_xpath('//th[.="Date (dd/mm/yyyy)"]')
        startup_name=head_tags.find_element_by_xpath('//th[.="Startup Name"]')
        industry_vertical=head_tags.find_element_by_xpath('//th[.="Industry / Vertical"]')
        sub_vertical=head_tags.find_element_by_xpath('//th[.="Sub-Vertical"]')
        
        city_location=head_tags.find_element_by_xpath('//th[.="City / Location"]')
        investors_name=head_tags.find_element_by_xpath('//th[.="Investors\' Name"]')
        investment_type=head_tags.find_element_by_xpath('//th[.="Investment Type"]')
        amount=head_tags.find_element_by_xpath('//th[.="Amount (In USD)"]')
        
        tr_rows_list=table.find_elements_by_xpath('.//tbody/tr')
        
        for tr in tr_rows_list:
            xpath_date='.//td[position()='+str(child.index(date)+1)+']'
            xpath_startup_name='.//td[position()='+str(child.index(startup_name)+1)+']'
            xpath_industry_vertical='.//td[position()='+str(child.index(industry_vertical)+1)+']'
            xpath_sub_vertical='.//td[position()='+str(child.index(sub_vertical)+1)+']'
            xpath_city_location='.//td[position()='+str(child.index(city_location)+1)+']'
            xpath_investors_name='.//td[position()='+str(child.index(investors_name)+1)+']'
            xpath_investment_type='.//td[position()='+str(child.index(investment_type)+1)+']'
            xpath_amount='.//td[position()='+str(child.index(amount)+1)+']'
            try:
                date_list.append(tr.find_element_by_xpath(xpath_date).text)
            except:
                date_list.append('-')
            try:
                startup_list.append(tr.find_element_by_xpath(xpath_startup_name).text)
            except:
                startup_list.append('-')
            try:
                industry_vertical_list.append(tr.find_element_by_xpath(xpath_industry_vertical).text)
            except:
                industry_vertical_list.append('-')
            try:
                sub_vertical_list.append(tr.find_element_by_xpath(xpath_sub_vertical).text)
            except:
                sub_vertical_list.append('-')
            try:
                city_location_list.append(tr.find_element_by_xpath(xpath_city_location).text)
            except:
                city_location_list.append('-')
            try:
                investors_name_list.append(tr.find_element_by_xpath(xpath_investors_name).text)
            except:
                investors_name_list.append('-')
            try:
                investment_type_list.append(tr.find_element_by_xpath(xpath_investment_type).text)
            except:
                investment_type_list.append('-')
            try:
                amount_list.append(tr.find_element_by_xpath(xpath_amount).text)
            except:
                amount_list.append('-')
    res_table=pd.DataFrame({'Date':date_list,'Startup Name':startup_list,'Industry / Vertical':industry_vertical_list,
                           'Sub-Vertical':sub_vertical_list,'City / Location':city_location_list,"Investors' Name":investors_name_list,
                           'Investment Type':investment_type_list,'Amount (In USD)':amount_list})
        
    return res_table
   

In [ ]:
get_funding_deals()

7. Write a program to scrap all the available details of best gaming laptops from digit.in.

In [ ]:
def get_gaming_laptops():
    #intiating the browser
    options = Options()
    options.add_argument("--disable-notifications")
    
    driver=webdriver.Chrome('C:/Users/yn/Desktop/Yuvi/DataTrained/seleniumWebdriver/chromedriver.exe',chrome_options=options)
    #loading the given url
    driver.get('https://www.digit.in/')
    #Send input to search bar
    time.sleep(5)
    driver.maximize_window()

    url=driver.find_element_by_xpath('//a[.="Laptops"]').get_attribute('href')
    driver.get(url)
    time.sleep(3)
    
    driver.find_element_by_xpath('//a[.="Best Gaming Laptops in India"]').click()
    time.sleep(3)
    prod_list=[]
    sell_list=[]
    price_list=[]
         
    head_tr=driver.find_element_by_xpath('//table[@id="summtable"]/thead/tr')
    child=head_tr.find_elements_by_xpath('//th')
    prod_name=head_tr.find_element_by_xpath('//th[.="Product Name"]')
    seller=head_tr.find_element_by_xpath('//th[.="Seller"]')
    price=head_tr.find_element_by_xpath('//th[.="Price"]')

    tr_rows=driver.find_elements_by_xpath('//table[@id="summtable"]/tbody/tr')
    for tr in tr_rows:
        xpath_prod_name='.//td[position()='+str(child.index(prod_name)+1)+']'
        xpath_seller='.//td[position()='+str(child.index(seller)+1)+']'
        xpath_price='.//td[position()='+str(child.index(price)+1)+']'

        try:
            prod_list.append(tr.find_element_by_xpath(xpath_prod_name).text)
        except:
            prod_list.append('-')
        try:
            sell_list.append(tr.find_element_by_xpath(xpath_seller).text)
        except:
            sell_list.append('-')
        try:
            price_list.append(tr.find_element_by_xpath(xpath_price).text)
        except:
            price_list.append('-')
    res_table=pd.DataFrame({'Product Name':prod_list,'Seller':sell_list,'Price':price_list})
    return res_table
            

In [ ]:
get_gaming_laptops()

8. Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be
scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.


In [ ]:
def get_billionaires():
    #intiating the browser
    options = Options()
    options.add_argument("--disable-notifications")
    
    driver=webdriver.Chrome('C:/Users/yn/Desktop/Yuvi/DataTrained/seleniumWebdriver/chromedriver.exe',chrome_options=options)
    #loading the given url
    driver.set_page_load_timeout(180)
    driver.get('https://www.forbes.com/')
    
    #Send input to search bar
    driver.maximize_window()
    WebDriverWait(driver,60).until(EC.presence_of_element_located((By.XPATH,'//div[@class="footer__nav"]/a[.="Billionaires"]')))
    driver.find_element_by_xpath('//div[@class="footer__nav"]/a[.="Billionaires"]').click()
    WebDriverWait(driver,60).until(EC.presence_of_element_located((By.XPATH,'//a[@aria-label="The World\'s Billionaires List 2021"]')))
    driver.find_element_by_xpath('//a[@aria-label="The World\'s Billionaires List 2021"]').click()
    WebDriverWait(driver,60).until(EC.presence_of_element_located((By.XPATH,'//div[@class="table-row-group"]/div[@class="table-row " or @class="table-row expanded"]')))
    rank_list=[]
    name_list=[]
    net_worth_list=[]
    age_list=[]
    city_list=[]
    source_list=[]
    ind_list=[]
    
    list_rows=driver.find_elements_by_xpath('//div[@class="table-row-group"]/div[@class="table-row " or @class="table-row expanded"]')
    
    for row in list_rows:
        try:            
            rank_list.append(row.find_element_by_xpath('.//div[@class="rank"]').text.replace('.',''))
        except:
            rank_list.append('-')
        try:
            name_list.append(row.find_element_by_xpath('.//div[@class="personName"]').text)
        except:
            name_list.append('-')
        try:
            net_worth_list.append(row.find_element_by_xpath('.//div[@class="netWorth"]').text)
        except:
            net_worth_list.append('-')
        try:
            age_list.append(row.find_element_by_xpath('.//div[@class="age"]').text)
        except:
            age_list.append('-')
        try:
            city_list.append(row.find_element_by_xpath('.//div[@class="countryOfCitizenship"]').text)
        except:
            city_list.append('-')
        try:
            source_list.append(row.find_element_by_xpath('.//div[@class="source"]').text)
        except:
            source_list.append('-')
        try:
            ind_list.append(row.find_element_by_xpath('.//div[@class="category"]').text)
        except:
            ind_list.append('-')
    res_table=pd.DataFrame({'Rank':rank_list,'Name':name_list,'Net worth':net_worth_list,'Age':age_list,'Citizenship':city_list,'Source':source_list,'Industry':ind_list})
    return res_table
    
    

In [ ]:
get_billionaires()

9. Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted
from any YouTube Video.

In [ ]:
def get_youtube_comments():
    #intiating the browser
    options = Options()
    options.add_argument("--disable-notifications")
    
    driver=webdriver.Chrome('C:/Users/yn/Desktop/Yuvi/DataTrained/seleniumWebdriver/chromedriver.exe',chrome_options=options)
    #loading the given url
    driver.get('https://www.youtube.com/watch?v=LHBE6Q9XlzI')
    driver.maximize_window()
    time.sleep(10)
    content_list=[]
    up_vote_list=[]
    time_list=[]
    driver.execute_script("window.scrollBy(0,500)","")
    i=0
    while(i<100):
        driver.execute_script("window.scrollBy(0,1500)","")
        time.sleep(10)
        i+=1
        
    comments_list=driver.find_elements_by_xpath('//div[@id="contents"]/ytd-comment-thread-renderer[@class="style-scope ytd-item-section-renderer"]')
    for com in comments_list:
        try:
            content_list.append(com.find_element_by_xpath('.//yt-formatted-string[@id="content-text"]').text)
        except:
            content_list.append('-')
        try:
            up_vote_list.append(com.find_element_by_xpath('.//span[@id="vote-count-middle"]').text)
        except:
            up_vote_list.append('-')
        try:
            time_list.append(com.find_element_by_xpath('.//yt-formatted-string[@class="published-time-text above-comment style-scope ytd-comment-renderer"]/a').text)
        except:
            time_list.append('-')
    res=pd.DataFrame({'Comment':content_list,'UpVotes':up_vote_list,'Posted Time':time_list})
    print(len(content_list))     
    return res
    

In [ ]:
get_youtube_comments()

10. Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in
“London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews,
overall reviews, privates from price, dorms from price, facilities and property description.

In [ ]:
def get_hostel_info():
    #intiating the browser
    options = Options()
    options.add_argument("--disable-notifications")
    
    driver=webdriver.Chrome('C:/Users/yn/Desktop/Yuvi/DataTrained/seleniumWebdriver/chromedriver.exe',chrome_options=options)
    #loading the given url
    driver.get('https://www.hostelworld.com/')
    driver.maximize_window()
    WebDriverWait(driver,60).until(EC.presence_of_element_located((By.XPATH,'//input[@class="location-text"]')))
    driver.find_element_by_xpath('//a[@title="Hostels"]').click()
    time.sleep(5)
    driver.find_element_by_xpath('//input[@id="home-search-keywords"]').send_keys('London')
    time.sleep(2)
    driver.find_element_by_xpath('//ul[@class="suggestions"]/li[@class="hover"]').click()
    driver.find_element_by_xpath('//button[.="Search"]').click()
    time.sleep(3)
       
    host_name=[]
    dist_list=[]
    rating_list=[]
    t_reviews_list=[]
    o_reviews_list=[]
    privates_price_list=[]
    dorms_price_list=[]
    facilty_list=[]
    desc_list=[]
    is_next=True
    WebDriverWait(driver,120).until(EC.presence_of_element_located((By.XPATH,'//div[@class="pagination-item pagination-next"]')))
   
    while(is_next):
        url_list=[]
        WebDriverWait(driver,60).until(EC.presence_of_element_located((By.XPATH,'//div[@class="page-inner"]//div[@class="prices-col"]/ancestor::div[@class="property-card"]')))
        prop_list=driver.find_elements_by_xpath('//div[@class="page-inner"]//div[@class="prices-col"]/ancestor::div[@class="property-card"]')
        for prop in prop_list:
            try:
                host_name.append(prop.find_element_by_xpath('.//h2[1]').text)
            except:
                host_name.append('-')
            try:
                dist_list.append(prop.find_element_by_xpath('.//span[@class="description"]').text)
            except:
                dist_list.append('-')
            try:
                rating_list.append(prop.find_element_by_xpath('.//div[contains(@class,"score")]').text)
            except:
                rating_list.append('-')
            try:
                t_reviews_list.append(prop.find_element_by_xpath('.//div[@class="reviews"]').text)
            except:
                t_reviews_list.append('-')
            try:
                o_reviews_list.append(prop.find_element_by_xpath('.//div[@class="keyword"]').text)
            except:
                o_reviews_list.append('-')
            try:
                privates_price_list.append(prop.find_element_by_xpath('.//p[.="Privates From"]/following-sibling::div[1]').text)
            except:
                privates_price_list.append('-')
            try:
                dorms_price_list.append(prop.find_element_by_xpath('.//p[.="Dorms From"]/following-sibling::div[1]').text)
            except:
                dorms_price_list.append('-')
            try:
                fac_info=''
                facilty_list_item=prop.find_elements_by_xpath('.//div[@class="facilities-label facilities"]/div')                
                for item in facilty_list_item:
                    fac_info=fac_info+', '+item.text
                facilty_list.append(fac_info[2:])
            except:
                facilty_list.append('-')
            try:            
                url=prop.find_element_by_xpath('.//h2[1]/a').get_attribute('href')
                url_list.append(url)               
            except:
                url_list.append('-')
        driver.execute_script("window.open('');")
        driver.switch_to.window(driver.window_handles[1])
        for prod in url_list:
            try:
                driver.get(prod)
                time.sleep(4)
                driver.find_element_by_xpath('//a[@class="toggle-content"]').click()
                time.sleep(2)
                WebDriverWait(driver,30).until(EC.presence_of_element_located((By.XPATH,'(//div[@class="description-container"])')))
                if(len(driver.find_elements_by_xpath('//div[@class="description-container"]'))>1):
                    desc_list.append(prop.find_element_by_xpath('(//div[@class="description-container"])[2]//div[@class="content"]').text)
                else:
                    desc_list.append(prop.find_element_by_xpath('(//div[@class="initial-property-info"])[1]').text)
            except:
                desc_list.append('-')
        # Closing new_url tab
        driver.close()
        # Switching to old tab
        driver.switch_to.window(driver.window_handles[0])       
        try:
            driver.refresh()
            time.sleep(5)           
            driver.find_element_by_xpath('//div[@class="pagination-item pagination-next"]').click()
        except NoSuchElementException:
            is_next=False
    res_table=pd.DataFrame({'Hostel Name':host_name,'Distance from city centre':dist_list,'Ratings':rating_list,'Total Reviews':t_reviews_list,'Overall review':o_reviews_list,
                           'Privates from price':privates_price_list,'Dorms from price':dorms_price_list,'Facilities':facilty_list,'Property Description':desc_list})
    return res_table

In [ ]:
get_hostel_info()